In [1]:
import sys
import os
import numpy as np
import pandas as pd
from glob import glob
from scipy.signal import ricker
from scipy.optimize import minimize

In [2]:
sys.path.append('../../../repos/seismiqb/')

In [5]:
from seismiqb import Well, OptimizationMixin, ImpulseOptimizationFactory, show_wavelet, symmetric_wavelet_estimation
from seismiqb import plot, Field

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## Parameters

In [ ]:
CUBE_PATH = '/data/seismic_data/seismic_interpretation/105_ZAPZIP/105_ZAPZIM_CLEANED.sgy'
WELL_PATH = '/data/seismic_data/seismic_interpretation/105_ZAPZIP/Well/LAS/*'
WELL_COORDINATES_PATH = '/data/seismic_data/seismic_interpretation/105_ZAPZIP/Well/Coordinates/Coordinates_wells.txt'


# Number of well-tie/wavelet-optimization pairs
N_ITERS = 4

# Ticks
CLEAN_TIME_SLICE = slice(346, 1219, None)
GLOBAL_SHIFT = 0.69130848
BOUNDS_MULTIPLIERS = [0.96, 1.04]

# Wavelet
CUT_FREQUENCY = 8
DELTA = .9

## Open and filter log

In [16]:
# Load info about the cube
field = Field(CUBE_PATH)
geometry = field.geometry
field.geometries = geometry

# Load the well
coordinates_df = pd.read_csv(COORDINATES_PATH, sep='\s+', encoding='1251')
coordinates = coordinates_df[['X_utm42', 'Y_utm42']].values
ordinal_coordinates = geometry.cdp_to_lines(coordinates).astype(np.int32) - geometry.shifts
name_to_coordinate = dict(zip(coordinates_df['N_skv'], ordinal_coordinates))

well_paths = sorted(glob(WELL_PATH))
well_name = os.path.basename(well_paths[3])
well_coordinates = name_to_coordinate[well_name].tolist()
well_slice = (*well_coordinates, slice(0, geometry.shape[-1]))
well = Well(storage=well_path, field=field)


# Recompute and filter impedance
nan_mask = np.isnan(well.DT) | np.isnan(well.RHOB)
well.RHOB = well.RHOB.fillna(method='bfill').fillna(method='ffill')
well.DT = well.DT.fillna(method='bfill').fillna(method='ffill')

well['DT_FILTERED'] = well.lowpass_filter(well.DT, pad_width=None, forward_backward=True)
well.microseconds_foot_to_seconds_meter(sonic_log='DT_FILTERED', name='DT_SECONDS_METER')

well['RHOB_FILTERED'] = well.lowpass_filter(well.RHOB, pad_width=0, forward_backward=True)
well.gramm_centimeter3_to_kilogramm_meter3(density_log='RHOB_FILTERED', name='RHOB_FILTERED')

well.compute_impedance_log_in_pascal_meter(sonic_log='DT_SECONDS_METER', density_log='RHOB_FILTERED')
well.pascal_meter_to_kilopascal_meter(name='AI_FILTERED')

well.AI_FILTERED[nan_mask] = np.nan


# Get seismic data and seismic time
seismic_time = np.arange(0.0, geometry.depth * geometry.sample_interval * 1e-3, geometry.sample_interval * 1e-3)
seismic_curve = geometry[well_slice]


# Compute initial well-time and wavelet
well_time_0 = np.cumsum(well.DT_FILTERED.values * 1e-6) + GLOBAL_SHIFT
raw_impedance = well['AI_FILTERED'].values
wavelet_0 = symmetric_wavelet_estimation(seismic_curve, wavelet_length=60)

## Run iterations of [well_tie + wavelet-optimization]

# Start well time and wavelet.

In [ ]:
well_time = well_time_0
wavelet = wavelet_0

for _ in range(N_ITERS):
    # Optimize and update well-time ticks
    results = well.optimize_well_time(well_time, seismic_time[CLEAN_TIME_SLICE],
                                      seismic_curve[CLEAN_TIME_SLICE], raw_impedance, impulse=wavelet[::-1].copy(),
                                      dt_bounds_multipliers=BOUNDS_MULTIPLIERS, n_iters=6000, optimizer='SGD',
                                      optimizer_kwargs={'lr': 2e-9})
    well_time, loss_history = results

    # Optimize and update wavelet
    factory = ImpulseOptimizationFactory(wavelet, seismic_time[CLEAN_TIME_SLICE], well_time,
                                         seismic_curve[CLEAN_TIME_SLICE], raw_impedance,
                                         cut_frequency=cut_frequency, delta=DELTA)

    results = minimize(factory, factory.get_x0, bounds=factory.get_bounds)
    wavelet = optimization_factory.compute_impulse(results['x'])

## Dump whatever needed

In [ ]:
# dump well_time, wavelet in whatever shape you want